In [1]:
%matplotlib inline

import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.stats import pearsonr

from scipy.cluster.hierarchy import linkage
import scipy.spatial.distance as ssd
from matplotlib.pyplot import rc_context

import seaborn as sns
import sys
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_selection import mutual_info_classif
import os
from cnmf import cNMF, load_df_from_npz
import harmonypy
sys.path.append('../../../Code/')
# from utils import TCAT

from sklearn import metrics
from utils import read_dataset_log
import harmonypy as hm
from sklearn.preprocessing import MinMaxScaler
from adjustText import adjust_text


In [2]:
! pwd

/data/srlab1/TCAT/Analysis/PerDataset/Sparks


In [3]:
params = read_dataset_log('Dataset Paths')
params.index = params['dataset']
params

,dataset,usage_fn,gene_scores_fn,gene_tpm_fn,tcat_fn,tcat_fn_withsingletons,manual_gating_fn,manual_activation_gating_fn,activation_score_fn,metadata_fn,...,k,dt,processed_forcnmf_fn,tpm_counts_for_cnmf_fn,raw_counts_filt_fn,raw_counts_fn,dataset_type,tissue_type,context_label,tcat_fn_old
dataset,,,,,,,,,,,,,,,,,,,,,
AMP-RA,AMP-RA,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMPRA....,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMPRA....,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMPRA....,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,/data/srlab1/TCAT/Data/PerDataset/AMPRA/Manual...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/Activa...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,...,34.0,0_15,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,discovery,Synovium,RA+OA,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
Pan-Cancer,Pan-Cancer,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Pa...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Pa...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Pa...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,NaN,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Ac...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,...,38.0,0.15,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,NaN,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,discovery,Pan-Tissue,Cancer+Healthy,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
TBRU,TBRU,/data/srlab1/TCAT/Data/PerDataset/TBRU/TBRU.20...,/data/srlab1/TCAT/Data/PerDataset/TBRU/TBRU.20...,/data/srlab1/TCAT/Data/PerDataset/TBRU/TBRU.20...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,/data/srlab1/TCAT/Data/PerDataset/TBRU/ManualG...,/data/srlab1/TCAT/Data/PerDataset/TBRU/Activat...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,...,36.0,0_20,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,discovery,Blood,Healthy,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
HIV-Vaccine,HIV-Vaccine,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/HIVV...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/HIVV...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/HIVV...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/Manu...,NaN,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/Acti...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,...,31.0,0_15,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,discovery,Blood,Post-Vaccine+Healthy,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
UK-Covid,UK-Covid,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/UKCO...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/UKCO...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/UKCO...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/Manu...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/Manu...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/Acti...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,...,44.0,0_20,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,NaN,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,discovery,Blood,Covid-19+Healthy,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
COMBAT,COMBAT,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/TCA

In [4]:
dataset = 'Sparks'


In [5]:
gating = pd.read_csv(params.loc[dataset, 'manual_gating_fn'], index_col = 0, sep = '\t',
                     header=0, names = ['cell_gating'])


In [6]:
params.loc[dataset, 'metadata_fn']

'/data/srlab1/TCAT/Data/PerDataset/Sparks2023/T_fromSeuratmetadata_RNAClustering_withSampID.tsv'

In [6]:
meta = pd.read_csv(params.loc[dataset, 'metadata_fn'], index_col = 0, sep = '\t')
meta['cell_gating'] = gating.loc[meta.index, 'cell_gating']
# meta = meta.loc[usage_norm.index, ]

cluster_col = params.loc[dataset, 'cluster_col']

/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
params.loc[dataset, 'metadata_fn']

'/data/srlab1/TCAT/Data/PerDataset/Sparks2023/T_fromSeuratmetadata_RNAClustering_withSampID.tsv'

In [8]:
meta.head(3)

,alt.subject.id,visit,covid.diagnosis.start.date.to.sample.drawn,group,sex,age,race,ethnicity,flu.vax.count.10yr,vaccine,...,leiden_res2_25,leiden_res2,leiden_res1_75,leiden_res1_5,leiden_res1_25,leiden_res0_75,leiden_res0_5,leiden_res0_25,alt.sample.id,cell_gating
Batch1_COVFLU_CITE_multi5P09_GCATGCGCAGCCTATA-1,Control,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5,5,5,4,0,0,0,0,Control,CD4_Naive
Batch1_COVFLU_CITE_multi5P02_TTGTAGGCAGTATCTG-1,Control,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,2,4,1,1,Control,CD4_CM
Batch1_COVFLU_CITE_multi5P02_GATGCTACAGCTTAAC-1,Control,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2,2,0,1,2,0,0,Control,CD4_Naive


In [7]:
pp_params = {
    'name_addition':None,
    'harmony_vars': ['alt.subject.id', 'visit', #'group', 
              'Batch'],
    'n_rna_pcs': 31,
    'n_neighbors': 30
}

if pp_params['name_addition'] is None:
    toadd = ''
else:
    toadd = pp_params['name_addition']


In [8]:
original =  '../../../Data/PerDataset/Sparks2023/T_fromSeurat.h5ad'

paths = {
        'Original' : original, # AnnData containing RNA +/- ADT,
        'RNA_LogTP10K_Varnorm_Harmony':original.replace('.h5ad', toadd + 'RNA.LogTP10K.Varnorm.Harmony.h5ad'),
        'Metadata_RNAClustering' : original.replace('.h5ad', toadd + 'metadata_RNAClustering.tsv')
        }
paths

{'Original': '../../../Data/PerDataset/Sparks2023/T_fromSeurat.h5ad',
 'RNA_LogTP10K_Varnorm_Harmony': '../../../Data/PerDataset/Sparks2023/T_fromSeuratRNA.LogTP10K.Varnorm.Harmony.h5ad',
 'Metadata_RNAClustering': '../../../Data/PerDataset/Sparks2023/T_fromSeuratmetadata_RNAClustering.tsv'}

# Preprocess and cluster RNA

In [6]:
adata = sc.read(params.loc[dataset, 'raw_counts_fn'])
ind = adata.var['feature_types']=='Antibody Capture'

In [54]:
params.loc[dataset, 'raw_counts_fn']

'/data/srlab1/TCAT/Data/PerDataset/Sparks2023/T_fromSeurat.h5ad'

In [7]:
adata_RNA = adata[:,~ind].copy()
sc.pp.normalize_per_cell(adata_RNA, counts_per_cell_after=1e4)
sc.pp.log1p(adata_RNA)
sc.pp.highly_variable_genes(adata_RNA)

In [8]:
sc.pp.scale(adata_RNA)
sc.tl.pca(adata_RNA, svd_solver='arpack')

In [11]:
ho = hm.run_harmony(adata_RNA.obsm['X_pca'][:, 0:pp_params['n_rna_pcs']], 
                    adata_RNA.obs, pp_params['harmony_vars'], random_state = 0)

2023-10-05 13:19:13,561 - harmonypy - INFO - Iteration 1 of 10
2023-10-05 13:23:36,825 - harmonypy - INFO - Iteration 2 of 10
2023-10-05 13:28:06,534 - harmonypy - INFO - Iteration 3 of 10
2023-10-05 13:32:24,932 - harmonypy - INFO - Converged after 3 iterations


In [12]:
adata_RNA.obsm['X_pca_harmony'] = ho.Z_corr.T

In [13]:
adata_RNA.obsm['X_pca_harmony'].shape, adata_RNA.obsm['X_pca'].shape

((336739, 31), (336739, 50))

In [15]:
sc.pp.neighbors(adata_RNA, n_neighbors=pp_params['n_neighbors'], 
                n_pcs=pp_params['n_rna_pcs'], use_rep='X_pca_harmony')

/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
sc.tl.umap(adata_RNA)

In [17]:
adata_RNA.obsm['X_harmony_RNA_UMAP'] = adata_RNA.obsm['X_umap']

In [ ]:
resolutions = [1, 0.8, 0.6, 2.25, 2, 1.75, 1.5, 1.25, 0.75, 0.5, 0.25]
for r in resolutions:
    print(r)
    sc.tl.leiden(adata_RNA, key_added='leiden_res%s' % str(r).replace('.', '_'), resolution = r)

In [76]:
paths['Metadata_RNAClustering']

'../../Data/PerDataset/Sparks2023/T_fromSeuratmetadata_RNAClustering.tsv'

In [78]:
paths['RNA_LogTP10K_Varnorm_Harmony']

'../../Data/PerDataset/Sparks2023/T_fromSeuratRNA.LogTP10K.Varnorm.Harmony.h5ad'

In [75]:
adata_RNA.obs.to_csv(paths['Metadata_RNAClustering'], sep='\t')

In [84]:
sc.write(paths['RNA_LogTP10K_Varnorm_Harmony'], adata_RNA)

Only considering the two last: ['.Harmony', '.h5ad'].
Only considering the two last: ['.Harmony', '.h5ad'].


# Assign gates to clusters

In [12]:
adata_RNA = sc.read(paths['RNA_LogTP10K_Varnorm_Harmony'])

Only considering the two last: ['.Harmony', '.h5ad'].
Only considering the two last: ['.Harmony', '.h5ad'].


In [13]:
adata_RNA

AnnData object with n_obs × n_vars = 336739 × 28402
    obs: 'alt.subject.id', 'visit', 'covid.diagnosis.start.date.to.sample.drawn', 'group', 'sex', 'age', 'race', 'ethnicity', 'flu.vax.count.10yr', 'vaccine', 'vaccine.side.effects', 'long.covid.symptoms', 'Batch', 'nCount_RNA', 'nCount_HTO', 'nCount_CITE', 'nCount_RNA_largest.gene', 'nCount_SCT', 'nFeature_RNA', 'nFeature_HTO', 'nFeature_CITE', 'nFeature_SCT', 'percent.mt', 'percent.largest.gene', 'coarse.cell.type', 'cell.type', 'HTO1', 'HTO2', 'HTO3', 'n_counts', 'leiden_res1', 'leiden_res0_8', 'leiden_res0_6', 'leiden_res2_25', 'leiden_res2', 'leiden_res1_75', 'leiden_res1_5', 'leiden_res1_25', 'leiden_res0_75', 'leiden_res0_5', 'leiden_res0_25'
    var: 'gene_ids', 'feature_types', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'leiden', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_harmony_RNA_UMAP', 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'd

In [14]:
paths['Metadata_RNAClustering']

'../../../Data/PerDataset/Sparks2023/T_fromSeuratmetadata_RNAClustering.tsv'

In [9]:
meta = pd.read_csv(paths['Metadata_RNAClustering'], index_col = 0, sep = '\t')

In [10]:
cluster_cols = list(meta.columns[meta.columns.str.startswith('leiden_res')])
cluster_cols

['leiden_res1',
 'leiden_res0_8',
 'leiden_res0_6',
 'leiden_res2_25',
 'leiden_res2',
 'leiden_res1_75',
 'leiden_res1_5',
 'leiden_res1_25',
 'leiden_res0_75',
 'leiden_res0_5',
 'leiden_res0_25']

In [15]:
gating['cell_gating'].unique()

array(['CD4_Naive', 'CD4_CM', 'CD8_EM', 'CD4_EM', 'Treg', 'CD8_TEMRA',
       'CD8_Naive', 'CD4_TEMRA', 'MAIT', 'gdT', 'DN', 'DP', 'CD8_CM'],
      dtype=object)

In [11]:
gates_to_exclude = ['CD4_TEMRA', 'DN', 'DP']
gate_list = list(set(gating['cell_gating'].unique()).difference(gates_to_exclude))

gate_list

['CD8_EM',
 'CD8_CM',
 'CD8_TEMRA',
 'CD4_CM',
 'MAIT',
 'CD4_EM',
 'gdT',
 'Treg',
 'CD8_Naive',
 'CD4_Naive']

In [12]:
# Filter out cells in other lineages
ind = gating[-gating['cell_gating'].isin(gates_to_exclude)].index
meta_test = meta.loc[ind, :]
gating_test = gating.loc[ind, :]

In [38]:
rna_clus_roc = pd.DataFrame(columns = ['gate', 'clusters', 'new_cluster', 
                                       'sensitivity', 'specificity', 
                                       'accuracy', 'delta_accuracy',
                                       'n_clusters', 'cluster_col'])
clus_assignments_all = {}

In [50]:
for rna_cluster_col in cluster_cols:
    print(rna_cluster_col)
    
    # Initialize for a given cluster resolution
    rna_clusters = meta_test[rna_cluster_col]

    i = 0 
    clus_assignments_all[rna_cluster_col] = dict(zip(gate_list, [[]]*len(gate_list)))
    assigned_clusters = [item for sublist in clus_assignments_all[rna_cluster_col].values() for item in sublist]
    accuracy_last = pd.Series(0.0, index = gate_list)
    iteration_loop = True
    gate_list_to_cluster = gate_list

    while(iteration_loop):
        # Loop through gates (only those that we got a positive delta_accuracy for last time)
        for gate_type in gate_list_to_cluster:
            gates = gating_test == gate_type

            # Loop through clusters, find ROC/AUC
            for cluster in set(rna_clusters.unique()).difference(assigned_clusters):
                pred = rna_clusters.isin(clus_assignments_all[rna_cluster_col][gate_type] + [cluster])
                fpr_rna_clus, tpr_rna_clus, _ = metrics.roc_curve(gates, pred, pos_label = True)
                accuracy = metrics.balanced_accuracy_score(gates, pred)
                rna_clus_roc = rna_clus_roc.append(pd.Series([gate_type, clus_assignments_all[rna_cluster_col][gate_type] + [cluster], cluster,
                                                                  tpr_rna_clus[1],  1 - fpr_rna_clus[1], 
                                                                  accuracy, accuracy - accuracy_last[gate_type],
                                                              i + 1, rna_cluster_col], 
                                                                 index=rna_clus_roc.columns), ignore_index=True)

        gate_loop = True
        iteration_loop = False
        # gate_list_to_cluster = []
        
        # Merge clusters with the highest delta_accuracy for a gate 
        while(gate_loop):
            assigned_clusters = [item for sublist in clus_assignments_all[rna_cluster_col].values() for item in sublist]
            possible_gates = [g for g, clus_list in clus_assignments_all[rna_cluster_col].items() if len(clus_list) < (i+1)]

            # Filter to this iteration, clusters not yet assigned, gates not assigned in this iteration, positive change in accuracy
            rna_clus_roc_subset = rna_clus_roc[(rna_clus_roc['cluster_col']==rna_cluster_col) &
                                                (-rna_clus_roc['new_cluster'].isin(assigned_clusters)) &
                                               (rna_clus_roc['n_clusters']==i+1) & 
                                               (rna_clus_roc['gate'].isin(possible_gates)) &
                                              (rna_clus_roc['delta_accuracy'] > 0)]

            # Update cluster assignments with the gate with largest delta_accuracy
            if rna_clus_roc_subset.shape[0] > 0:
                g_roc = rna_clus_roc_subset.sort_values('delta_accuracy', ascending = False).head(1)
                gate_type = g_roc['gate'].values[0]
                clus_assignments_all[rna_cluster_col][gate_type] = clus_assignments_all[rna_cluster_col][gate_type]+ [g_roc['new_cluster'].values[0]]
                accuracy_last[gate_type] = g_roc['accuracy'].values[0]

                iteration_loop = True # At least one of the gates may still benefit from further merging
                # gate_list_to_cluster = gate_list_to_cluster + [gate_type]

            else:
                gate_loop = False
        i+=1

leiden_res1
leiden_res0_8
leiden_res0_6
leiden_res2_25
leiden_res2
leiden_res1_75
leiden_res1_5
leiden_res1_25
leiden_res0_75
leiden_res0_5
leiden_res0_25


In [ ]:
for rna_cluster_col in cluster_cols:
    print(rna_cluster_col)
    
    # Initialize for a given cluster resolution
    rna_clusters = meta_test[rna_cluster_col]


In [51]:
rna_clus_roc['clusters_str'] = [','.join(map(str,clusters)) for clusters in rna_clus_roc['clusters']]

In [52]:
# Filter to best clustering results
clus_assignments_all_df = pd.DataFrame(columns = ['gate', 'clusters', 'cluster_col',
                                                 'sensitivity', 'specificity', 'accuracy'])

for rna_cluster_col in cluster_cols:
    print(rna_cluster_col)
    dat = pd.DataFrame(zip([gate for gate, clusters in clus_assignments_all[rna_cluster_col].items()],
                 [','.join(map(str,clusters)) for gate, clusters in clus_assignments_all[rna_cluster_col].items()],
                 [rna_cluster_col]*len(clus_assignments_all[rna_cluster_col])),
            columns =  clus_assignments_all_df.columns[0:3])
    dat = pd.merge(dat, rna_clus_roc[['gate', 'clusters_str', 'cluster_col', 'sensitivity', 'specificity', 'accuracy']], 
         left_on = ['gate', 'clusters', 'cluster_col'],
         right_on = ['gate', 'clusters_str', 'cluster_col']).drop('clusters_str', axis = 1)

    clus_assignments_all_df = pd.concat([clus_assignments_all_df, dat], ignore_index = True)

leiden_res1
leiden_res0_8
leiden_res0_6
leiden_res2_25
leiden_res2
leiden_res1_75
leiden_res1_5
leiden_res1_25
leiden_res0_75
leiden_res0_5
leiden_res0_25


In [53]:
clus_assignments_all

{'leiden_res1': {'CD4_EM': [1, 12],
  'CD4_Naive': [0, 2],
  'MAIT': [6],
  'Treg': [9, 13, 15],
  'gdT': [3],
  'CD4_CM': [4, 11],
  'CD8_CM': [10],
  'CD8_EM': [7, 14],
  'CD8_TEMRA': [8],
  'CD8_Naive': [5]},
 'leiden_res0_8': {'CD4_EM': [1, 12],
  'CD4_Naive': [0, 2, 10],
  'MAIT': [7],
  'Treg': [9],
  'gdT': [3],
  'CD4_CM': [4, 11],
  'CD8_CM': [13],
  'CD8_EM': [6, 14],
  'CD8_TEMRA': [8],
  'CD8_Naive': [5]},
 'leiden_res0_6': {'CD4_EM': [1, 11],
  'CD4_Naive': [0, 3],
  'MAIT': [7],
  'Treg': [9, 14],
  'gdT': [2],
  'CD4_CM': [4, 10],
  'CD8_CM': [12],
  'CD8_EM': [6, 13],
  'CD8_TEMRA': [8],
  'CD8_Naive': [5]},
 'leiden_res2_25': {'CD4_EM': [9, 12, 18, 24],
  'CD4_Naive': [2, 5, 7, 10, 8, 23, 26],
  'MAIT': [6, 22],
  'Treg': [13, 21, 28],
  'gdT': [15, 3],
  'CD4_CM': [1, 4, 20],
  'CD8_CM': [14, 25],
  'CD8_EM': [11, 16, 27],
  'CD8_TEMRA': [17, 19],
  'CD8_Naive': [0]},
 'leiden_res2': {'CD4_EM': [10, 12, 17],
  'CD4_Naive': [2, 4, 5, 9, 14, 22, 25],
  'MAIT': [7, 26],


## Save cluster assignments and stats

In [54]:
paths

{'Original': '../../../Data/PerDataset/Sparks2023/T_fromSeurat.h5ad',
 'RNA_LogTP10K_Varnorm_Harmony': '../../../Data/PerDataset/Sparks2023/T_fromSeuratRNA.LogTP10K.Varnorm.Harmony.h5ad',
 'Metadata_RNAClustering': '../../../Data/PerDataset/Sparks2023/T_fromSeuratmetadata_RNAClustering.tsv',
 'rna_clus_no_overlap': '../../../Data/PerDataset/Sparks2023/ROC_RNAClustering_MaxAccuracy_NoOverlap.tsv',
 'rna_clus_accuracy_no_overlap': '../../../Data/PerDataset/Sparks2023/ROC_RNAClustering_MaxAccuracy_NoOverlap.tsv'}

In [54]:
paths.update({
    'rna_clus_accuracy_no_overlap':original.replace('T_fromSeurat.h5ad', toadd + 'ROC_RNAClustering_MaxAccuracy_NoOverlap.tsv')

})
paths

{'Original': '../../../Data/PerDataset/Sparks2023/T_fromSeurat.h5ad',
 'RNA_LogTP10K_Varnorm_Harmony': '../../../Data/PerDataset/Sparks2023/T_fromSeuratRNA.LogTP10K.Varnorm.Harmony.h5ad',
 'Metadata_RNAClustering': '../../../Data/PerDataset/Sparks2023/T_fromSeuratmetadata_RNAClustering.tsv',
 'rna_clus_accuracy_no_overlap': '../../../Data/PerDataset/Sparks2023/ROC_RNAClustering_MaxAccuracy_NoOverlap.tsv'}

In [56]:
clus_assignments_all_df.to_csv(paths['rna_clus_accuracy_no_overlap'], sep='\t')

In [57]:
clus_assignments_all_df

,gate,clusters,cluster_col,sensitivity,specificity,accuracy
0,CD4_EM,"1,12",leiden_res1,0.586381,0.917580,0.751980
1,CD4_Naive,"0,2",leiden_res1,0.896491,0.962039,0.929265
2,MAIT,6,leiden_res1,0.938998,0.988568,0.963783
3,Treg,"9,13,15",leiden_res1,0.769948,0.979343,0.874646
4,gdT,3,leiden_res1,0.794967,0.915318,0.855143
...,...,...,...,...,...,...
105,CD4_CM,1,leiden_res0_25,0.775776,0.862135,0.818956
106,CD8_CM,8,leiden_res0_25,0.007904,0.997286,0.502595
107,CD8_EM,"2,9",leiden_res0_25,0.869725,0.880924,0.875325
108,CD8_TEMRA,5,leiden_res0_25,0.071184,0.958000,0.514592


In [58]:
clus_assignments_all_df[clus_assignments_all_df['gate']=='CD4_Naive']

,gate,clusters,cluster_col,sensitivity,specificity,accuracy
1,CD4_Naive,"0,2",leiden_res1,0.896491,0.962039,0.929265
11,CD4_Naive,"0,2,10",leiden_res0_8,0.903238,0.959397,0.931318
21,CD4_Naive,"0,3",leiden_res0_6,0.903331,0.960627,0.931979
31,CD4_Naive,"2,5,7,10,8,23,26",leiden_res2_25,0.881261,0.951476,0.916368
41,CD4_Naive,"2,4,5,9,14,22,25",leiden_res2,0.880652,0.953481,0.917066
51,CD4_Naive,"2,4,5,9,17,22",leiden_res1_75,0.888587,0.952326,0.920456
61,CD4_Naive,"0,4,5,10",leiden_res1_5,0.889083,0.954488,0.921786
71,CD4_Naive,"0,1,9",leiden_res1_25,0.891356,0.956494,0.923925
81,CD4_Naive,"0,2",leiden_res0_75,0.902350,0.960688,0.931519
91,CD4_Naive,"0,10",leiden_res0_5,0.916494,0.956720,0.936607
